In [83]:
# -*- coding: utf-8 -*-
# web_site（https://www.kaggle.com/c/avazu-ctr-prediction）

import pandas as pd
import numpy as np

#读取50万行数据
df = pd.read_csv('rawdata/train.csv',nrows=500000)
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [84]:
df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [85]:
df['banner_pos'].value_counts()

0    393983
1    105269
5       538
2       148
7        45
4        17
Name: banner_pos, dtype: int64

In [86]:
df['device_conn_type'].value_counts()

0    447909
2     45337
3      6313
5       441
Name: device_conn_type, dtype: int64

In [87]:
data = df.drop(['id'],axis=1)

In [88]:
for feature in data.columns:
    # 将字符串编码为数字特征
    data[feature]=pd.factorize(data[feature])[0]

In [89]:
data.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,1,1,1,0,0,0,...,0,1,2,0,0,1,0,0,0,1


In [90]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(data, test_size=0.2)

In [91]:
import lightgbm as lgb


clf=lgb.LGBMClassifier(is_unbalanced=False)
cal = list(train.drop('click',axis=1).columns)
#训练
clf.fit(train.drop('click',axis=1),train['click'],categorical_feature=cal)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', is_unbalanced=False, learning_rate=0.1,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [92]:
predict = clf.predict_proba(test.drop('click',axis=1))[:,1]

In [93]:
def celoss(target, predict):
    target = np.array(target)
    predict = np.array(predict)
    return -(target*np.log(predict) + (1-target)*np.log(1-predict)).mean()

In [94]:
celoss(test['click'],predict)

0.37987165096116166